In [1]:
import numpy as np
import math

In [2]:
import inspect, re

def varname(p):
    for line in inspect.getframeinfo(inspect.currentframe().f_back)[3]:
        m = re.search(r'\bvarname\s*\(\s*([A-Za-z_][A-Za-z0-9_]*)\s*\)', line)
        if m:
              return m.group(1)
            
def namestr(obj, namespace):
    return [name for name in namespace if namespace[name] is obj]

In [3]:
from cardsample_problem import *

|S| = 2314
|O| = 543


In [4]:
init_belief_hist = init_belief_histogram(all_states)

print(init_belief_hist)

[(So0I0I0I0I0I0I1I0I0oC2RNoneT0, 0.3333333333333333), (So1I0I0I0I0I0I0I0I0oC0RNoneT0, 0.3333333333333333), (So0I0I0I1I0I0I0I0I0oC1RNoneT0, 0.3333333333333333), (So2I2I1I2I2I1I2I1I0oC2R0T0, 0.0), (So2I1I0I0I1I0I1I0I0oC0R0T0, 0.0)]


In [6]:
def print_state(s):
    print(namestr(s, globals())[0], s.val, s.card, s.r, s.terminal)
    
    
s1 = State((1,0,0,0,1,0,2,1,0), 2, r=0, terminal=False)
print(s1)

a1 = Action(0)

s2 = card_problem.agent.transition_model.sample(s1,  a1)
prob2 = card_problem.agent.transition_model.probability(s2, s1, a1)
print(s2, prob2, a1)

So1I0I0I0I1I0I2I1I0oC2R0T0
So1I1I0I1I1I0I2I1I0oC1R0T0 0.6666666666666666 A0


In [8]:
T = P.m*P.n
init_true_state= get_random_state()
init_belief_hist = init_belief_histogram(all_states)
init_belief_part = init_belief_particle_s0(all_states)
card_problem = CardProblem(init_true_state, init_belief_hist)
#all_actions = [Action(a) for a in range(P.n)]
        
# State transition probabilities - they need to be normalized
for s in list(card_problem.agent.all_states):
    for a in list(card_problem.agent.all_actions):
        probs = []
        if s == s1:
            print("s == s1", a)
            s3 = card_problem.agent.transition_model.sample(s,  a)
            print_state(s); print_state(s1); print_state(s3) 
        for s_next in list(card_problem.agent.all_states):
            prob = card_problem.agent.transition_model.probability(s_next, s, a)
            probs.append(prob)
            if s == s1 and s_next == s3 and s == s1:                
                prob3 = card_problem.agent.transition_model.probability(s3, s, a)
                print("prob for s3,s,a", prob3)
                prob2 = card_problem.agent.transition_model.probability(s_next, s, a)
                print("prob for s_next,s,a", prob2)
            if s == s1 and s_next == s2 and a == a1:
                print_state(s); print_state(s1)
                print_state(s_next); print_state(s2)
                prob2 = card_problem.agent.transition_model.probability(s2, s1, a1)
                
                prob3 = card_problem.agent.transition_model.probability(s_next, s, a)
                print(prob, prob2, prob3, a1)
        total_prob = sum(probs)
        if total_prob == 0:
            print_state(s)
            print(a)
        for i, s_next in enumerate(all_states):
            prob_norm = probs[i] / total_prob
            #print('T : %s : %s : %s %f\n' % (a, s, s_next, prob_norm))

KeyboardInterrupt: 

In [ ]:
print_state(s), print_state(s_next)

In [ ]:
for a in range(P.n):
    print(card_problem.agent.tree[a], card_problem.agent.tree)
    print(tree_i[a])

In [ ]:
(P.n*2)**(P.n*P.m)

In [ ]:
len(all_states)

In [ ]:
# states for n=5;m=2 is 330438

In [ ]:
for s in all_states:
    val = np.array(s.val)
    if (val[::3].sum()-1 != val[1::3].sum()) and s.card != "$" :
        print(s, val[::3].sum(), val[1::3].sum(),"error in states actions")

In [ ]:
all_obs = create_all_observations(all_states)

In [ ]:
len(all_obs)

In [10]:
rewards_all = np.load("rewards_1by11000_m2n3.npy")

num = (rewards_all >= 0).sum()
print(num, rewards_all[:num].sum() * 1.0 / num)
#print(rewards_all)

1000 2.0


In [11]:
rewards_all = np.load("rewards_rand1000_m2n3.npy")

num = (rewards_all >= 0).sum()
print(num, rewards_all[:num].sum() * 1.0 / num)

#print(rewards_all)

1000 2.06


In [17]:
rewards_all = np.load("rewards_vi1000_T6.npy")

num = (rewards_all >= 0).sum()
print(num)

print(rewards_all[:num].sum() * 1.0 / num)

print(rewards_all)

561
2.9893048128342246
[  4.   2.   2.   4.   1.   2.   4.   2.   4.   5.   4.   4.   2.   2.
   1.   2.   4.   1.   2.   4.   4.   4.   4.   2.   4.   4.   4.   4.
   1.   4.   1.   1.   4.   4.   2.   2.   1.   4.   4.   1.   4.   1.
   4.   4.   1.   4.   4.   4.   4.   1.   5.   1.   1.   4.   4.   4.
   2.   4.   1.   5.   2.   4.   4.   4.   4.   2.   2.   4.   4.   1.
   4.   5.   4.   2.   5.   1.   4.   1.   2.   4.   5.   4.   2.   4.
   4.   4.   4.   1.   4.   4.   4.   4.   4.   2.   4.   4.   1.   4.
   2.   2.   4.   1.   1.   5.   4.   1.   4.   4.   1.   4.   4.   4.
   4.   2.   4.   4.   2.   1.   4.   4.   1.   4.   1.   4.   4.   1.
   2.   4.   1.   4.   4.   2.   4.   2.   4.   4.   4.   2.   5.   4.
   2.   1.   4.   4.   4.   4.   4.   5.   2.   1.   4.   2.   4.   2.
   1.   4.   2.   2.   4.   4.   4.   2.   1.   1.   4.   4.   2.   1.
   4.   2.   1.   1.   4.   4.   4.   2.   4.   4.   4.   4.   5.   4.
   1.   4.   4.   2.   4.   4.   4.   4.   4.   1.   4

In [18]:
rewards_all = np.load("rewards_pouct1000_m2n2.npy")

num = (rewards_all >= 0).sum()
print(num)

print(rewards_all[:num].sum() * 1.0 / num)

print(rewards_all)

1000
2.857
[3. 3. 3. 3. 4. 2. 3. 3. 2. 3. 4. 3. 2. 3. 2. 3. 3. 2. 2. 2. 3. 3. 3. 3.
 3. 4. 2. 3. 2. 3. 2. 4. 3. 3. 2. 4. 3. 3. 4. 4. 3. 3. 4. 3. 3. 4. 3. 3.
 3. 2. 3. 3. 3. 2. 4. 3. 4. 3. 2. 3. 3. 3. 2. 3. 3. 3. 4. 2. 2. 3. 2. 3.
 3. 4. 2. 3. 4. 3. 2. 3. 3. 3. 2. 2. 2. 3. 3. 3. 4. 3. 2. 3. 2. 2. 2. 3.
 3. 3. 3. 3. 4. 3. 3. 3. 3. 4. 3. 2. 4. 2. 2. 3. 3. 3. 3. 3. 3. 2. 2. 2.
 2. 2. 2. 2. 4. 3. 3. 3. 3. 2. 2. 3. 4. 3. 3. 2. 3. 3. 3. 3. 2. 3. 3. 3.
 3. 3. 4. 4. 3. 3. 3. 3. 4. 3. 2. 2. 3. 3. 3. 3. 2. 4. 3. 3. 2. 4. 2. 3.
 4. 3. 2. 2. 3. 4. 2. 4. 4. 4. 3. 3. 2. 2. 4. 2. 2. 3. 3. 2. 4. 4. 2. 2.
 2. 2. 4. 3. 2. 3. 3. 3. 2. 3. 4. 3. 2. 3. 2. 3. 4. 3. 3. 2. 3. 2. 3. 3.
 3. 2. 3. 3. 3. 3. 3. 3. 3. 2. 3. 3. 4. 2. 4. 2. 3. 3. 2. 3. 4. 2. 2. 2.
 2. 2. 3. 4. 3. 4. 4. 3. 3. 2. 3. 2. 3. 3. 3. 2. 3. 3. 3. 2. 2. 2. 3. 4.
 2. 2. 2. 3. 3. 2. 3. 2. 4. 3. 3. 3. 2. 2. 3. 3. 3. 3. 3. 2. 3. 4. 3. 3.
 4. 3. 4. 4. 3. 3. 3. 2. 2. 3. 3. 4. 2. 3. 3. 3. 3. 4. 4. 3. 2. 2. 2. 4.
 3. 3. 3. 3. 3. 3. 4. 3. 3. 3. 4. 2. 4. 

In [ ]:
np.set_printoptions(precision=8)

In [67]:
rewards_all = np.load("rewards_pouct1000_m2n3.npy")

num = (rewards_all >= 0).sum()
print(num)

print(rewards_all[:num].sum() * 1.0 / num)

print(rewards_all)

221
2.97737556561086
[  2.   2.   2.   2.   2.   3.   3.   2.   3.   3.   2.   3.   2.   3.
   2.   4.   2.   2.   3.   3.   4.   3.   1.   2.   2.   2.   3.   3.
   4.   4.   4.   2.   3.   3.   4.   2.   6.   2.   2.   2.   3.   3.
   2.   4.   2.   4.   3.   3.   5.   2.   2.   2.   4.   3.   4.   4.
   3.   5.   3.   2.   2.   4.   3.   4.   4.   1.   3.   3.   2.   4.
   2.   2.   3.   3.   4.   2.   2.   3.   2.   2.   2.   2.   2.   3.
   1.   4.   4.   2.   2.   5.   4.   4.   4.   3.   5.   5.   5.   4.
   3.   4.   2.   3.   1.   6.   4.   3.   1.   3.   2.   3.   4.   3.
   3.   2.   5.   2.   1.   3.   3.   1.   2.   4.   3.   3.   4.   2.
   3.   3.   4.   3.   2.   4.   2.   3.   2.   2.   4.   4.   4.   2.
   2.   4.   2.   4.   3.   3.   4.   4.   2.   3.   2.   2.   2.   4.
   3.   1.   3.   5.   1.   2.   4.   4.   1.   4.   3.   3.   4.   2.
   5.   3.   5.   4.   2.   2.   3.   2.   2.   4.   2.   3.   4.   4.
   3.   3.   4.   2.   4.   3.   3.   3.   2.   4.   2. 

In [ ]:
card_problem.agent.tree.children

In [ ]:
rewards_all

In [ ]:
from julia.api import Julia
jl = Julia(compiled_modules=False)

from julia.QMDP import QMDPSolver
from julia.SARSOP import SARSOPSolver